In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from imblearn.over_sampling import SMOTE
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [5]:
def downsample_largest_class(X, y):
    """
    Downsample the largest class to the size of the second largest class.
    
    Args:
        X (pd.DataFrame or np.array): Features.
        y (pd.Series or np.array): Labels.
        
    Returns:
        tuple: Downsampled X and y.
    """
    # Count class frequencies
    class_counts = Counter(y)
    sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)
    largest_class, largest_count = sorted_classes[0]
    second_largest_count = sorted_classes[1][1]
    
    # Resample the largest class to the size of the second largest
    X_largest = X[y == largest_class]
    y_largest = y[y == largest_class]
    
    X_largest_downsampled, y_largest_downsampled = resample(
        X_largest, y_largest, replace=False, n_samples=second_largest_count, random_state=42
    )
    
    # Combine the downsampled largest class with the rest
    X_rest = X[y != largest_class]
    y_rest = y[y != largest_class]
    X_final = pd.concat([X_rest, X_largest_downsampled])
    y_final = pd.concat([y_rest, y_largest_downsampled])
    
    return X_final, y_final


def apply_smote_after_downsampling(X, y):
    """
    Downsample the largest class to the size of the second largest class,
    then apply SMOTE dynamically based on the smallest class size.
    
    Args:
        X (pd.DataFrame or np.array): Features.
        y (pd.Series or np.array): Labels.
        
    Returns:
        tuple: Resampled X and y.
    """
    # Downsample the largest class
    X, y = downsample_largest_class(X, y)
    
    # Determine n_neighbors dynamically based on the smallest class size
    class_counts = Counter(y)
    min_class_size = min(class_counts.values())
    n_neighbors = min(5, min_class_size - 1)  # Adjust neighbors to fit smallest class
    
    # Apply SMOTE
    smote = SMOTE(random_state=42, k_neighbors=n_neighbors)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    
    return X_resampled, y_resampled

In [6]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle

    # Resample Training Data
    X = train_data.iloc[:, :-1]
    y = train_data.iloc[:, -1]
    X_resampled, y_resampled = apply_smote_after_downsampling(X, y)

    # Prepare Training Data
    if (is_string_labels):
        y_resampled = y_resampled.map(label_mapping)
    train_X = X_resampled.to_numpy()
    train_y = y_resampled.to_numpy()
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))

    # Perform a stratified split into train and validation sets (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    print("Verbose output only for first run...")
    verbose_run = 1
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        input_layer = Input(shape=(X_train.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, X_train.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            X_train, y_train, 
            epochs=1000, 
            batch_size=int(X_train.shape[0] * 0.01), 
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")
        

In [7]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.5540 - loss: 1.3248 - val_accuracy: 0.9212 - val_loss: 0.3363
Epoch 2/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9128 - loss: 0.3222 - val_accuracy: 0.9569 - val_loss: 0.1321
Epoch 3/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9573 - loss: 0.1657 - val_accuracy: 0.9719 - val_loss: 0.0930
Epoch 4/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9699 - loss: 0.1169 - val_accuracy: 0.9747 - val_loss: 0.0785
Epoch 5/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9736 - loss: 0.0954 - val_accuracy: 0.9773 - val_loss: 0.0700
Epoch 6/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9753 - loss: 0.0834 - val_accuracy: 0.9786 - val_loss: 0.0662
Epoch 7/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9762 - loss: 0.0802 - val_accuracy: 0.9810 - val_loss: 0.0621
Epoch 8/1000
101/101 ━━━━━

In [8]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 25s 81ms/step - accuracy: 0.1780 - loss: 1.9235 - val_accuracy: 0.1902 - val_loss: 1.9075
Epoch 2/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - accuracy: 0.1880 - loss: 1.9120 - val_accuracy: 0.1943 - val_loss: 1.9054
Epoch 3/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - accuracy: 0.1901 - loss: 1.9102 - val_accuracy: 0.1966 - val_loss: 1.9045
Epoch 4/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.1921 - loss: 1.9082 - val_accuracy: 0.1944 - val_loss: 1.9039
Epoch 5/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.1925 - loss: 1.9087 - val_accuracy: 0.1954 - val_loss: 1.9035
Epoch 6/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.1921 - loss: 1.9082 - val_accuracy: 0.1942 - val_loss: 1.9027
Epoch 7/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.1931 - loss: 1.9070 - val_accuracy: 0.1972 - val_loss: 1.9024
Epoch 8/1000
101/101 ━━━━━

In [9]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.1160 - loss: 2.9315 - val_accuracy: 0.1956 - val_loss: 2.5011
Epoch 2/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.2058 - loss: 2.4977 - val_accuracy: 0.2791 - val_loss: 2.3376
Epoch 3/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.2533 - loss: 2.3466 - val_accuracy: 0.2885 - val_loss: 2.2464
Epoch 4/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.2766 - loss: 2.2608 - val_accuracy: 0.3072 - val_loss: 2.1758
Epoch 5/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.2917 - loss: 2.1891 - val_accuracy: 0.3120 - val_loss: 2.1098
Epoch 6/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.2975 - loss: 2.1403 - val_accuracy: 0.3167 - val_loss: 2.0657
Epoch 7/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.3023 - loss: 2.0968 - val_accuracy: 0.3190 - val_loss: 2.0378
Epoch 8/1000
101/101 ━━━━━

In [10]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.2210 - loss: 2.0153 - val_accuracy: 0.2893 - val_loss: 1.8739
Epoch 2/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2823 - loss: 1.8915 - val_accuracy: 0.3048 - val_loss: 1.8373
Epoch 3/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.3044 - loss: 1.8601 - val_accuracy: 0.3195 - val_loss: 1.8142
Epoch 4/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3083 - loss: 1.8377 - val_accuracy: 0.3127 - val_loss: 1.8009
Epoch 5/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3207 - loss: 1.8228 - val_accuracy: 0.3128 - val_loss: 1.8002
Epoch 6/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3148 - loss: 1.8200 - val_accuracy: 0.3249 - val_loss: 1.7854
Epoch 7/1000
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3242 - loss: 1.8080 - val_accuracy: 0.3283 - val_loss: 1.7797
Epoch 8/1000
101/101 ━━━━━